In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

sns.set(style="whitegrid")

df = pd.read_csv('dados.csv')

print(f"\nNúmero de linhas e colunas: {df.shape}\n")
print(tabulate(df.head(10), headers='keys', tablefmt='pretty'))
print(df.info())
print(df.describe())

# Limpeza dos dados
df['Released_Year'] = pd.to_numeric(df['Released_Year'], errors='coerce')
df.dropna(subset=['Released_Year'], inplace=True)
df['Released_Year'] = df['Released_Year'].astype(int)
df['Runtime'] = df['Runtime'].str.extract('(\d+)').astype(int)

# Histograma IMDB_Rating
plt.figure(figsize=(10,6))
sns.histplot(df['IMDB_Rating'], bins=20, kde=True)
plt.title('Distribuição das Avaliações IMDB')
plt.xlabel('IMDB Rating')
plt.ylabel('Número de Filmes')
plt.show()

df_genres = df.assign(Genre=df['Genre'].str.split(', ')).explode('Genre')

media_generos = df_genres.groupby('Genre')['IMDB_Rating'].mean().sort_values(ascending=False)

# Scatterplot
plt.figure(figsize=(15,6))
sns.scatterplot(data=df, x='Runtime', y='IMDB_Rating')
plt.title('Avaliação IMDB x Duração do Filme')
plt.xlabel('Duração (minutos)')
plt.ylabel('IMDB Rating')
plt.show()

# Gêneros bem avaliados
limite = 8  
generos_maiores_avaliacoes = media_generos[media_generos > limite]
print(f"\nGêneros com média de IMDB acima de {limite}:")
print(generos_maiores_avaliacoes)
print("Número de gêneros com maiores avaliações:", len(generos_maiores_avaliacoes))

if len(generos_maiores_avaliacoes) > 0:
    plt.figure(figsize=(10,6))
    sns.barplot(x=generos_maiores_avaliacoes.index, 
                y=generos_maiores_avaliacoes.values, 
                palette="Blues_r")
    plt.title(f'Gêneros com média de IMDB acima de {limite}', fontsize=16)
    plt.ylabel('Média IMDB', fontsize=12)
    plt.xlabel('Gênero', fontsize=12)
    plt.xticks(rotation=45)
    plt.ylim(generos_maiores_avaliacoes.min() - 0.1, generos_maiores_avaliacoes.max() + 0.1)
    plt.show()
else:
    print("Nenhum gênero acima do limite definido.")

series_top = df[df['IMDB_Rating'] > 8][['Series_Title', 'Released_Year', 'IMDB_Rating']]
print(tabulate(series_top.head(20), headers='keys', tablefmt='pretty'))
